# Data Wrangling - LendingClub Loan Listings Data

Meskerem Goshime

Data Source: https://www.kaggle.com/datasets/husainsb/lendingclub-issued-loans

“LendingClub is a US peer-to-peer lending company, headquartered in San Francisco, California.” The company “enables borrowers to create unsecured personal loans between 1,000 and 40,000 dollars. The standard loan period is three years. Investors can search and browse the loan listings on Lending Club website and select loans that they want to invest in based on the information supplied about the borrower, amount of loan, loan grade, and loan purpose. Investors make money from interest. Lending Club makes money by charging borrowers an origination fee and investors a service fee.” (source: https://www.kaggle.com/datasets/husainsb/lendingclub-issued-loans?select=lc_2016_2017.csv)​

This study aims to help LendingClub in make informed lending decisions and investors make informed investment decisions​

In [114]:
import pandas as pd
import numpy as np
import datetime as dt

In [115]:
# The data was downloaded as a csv file. 
# Data Source: https://www.kaggle.com/datasets/husainsb/lendingclub-issued-loans?select=lc_2016_2017.csv%29%E2%80%8B
# I am using the lc_loan file only, which contains loans issued from 2007 to 2015. 
# This dataset contains 887,379 rows and 74 columns.

# Read Loan_data from the CSV file.

Loan_data = pd.read_csv('../data/raw/lc_loan.csv', low_memory=False)

In [116]:
Loan_data.head(3)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
#Taking a look at the columns and their data types of the main Loan data

Loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887379 entries, 0 to 887378
Data columns (total 74 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           887379 non-null  int64  
 1   member_id                    887379 non-null  int64  
 2   loan_amnt                    887379 non-null  float64
 3   funded_amnt                  887379 non-null  float64
 4   funded_amnt_inv              887379 non-null  float64
 5   term                         887379 non-null  object 
 6   int_rate                     887379 non-null  float64
 7   installment                  887379 non-null  float64
 8   grade                        887379 non-null  object 
 9   sub_grade                    887379 non-null  object 
 10  emp_title                    835917 non-null  object 
 11  emp_length                   842554 non-null  object 
 12  home_ownership               887379 non-null  object 
 13 

In [118]:
# Checking if there are duplicates in the Loan_data

Duplicates = Loan_data.duplicated().sum()
Duplicates

# No duplicates.

0

In [119]:
#checking missing values in Loan_data

Missing = pd.DataFrame([Loan_data.isnull().sum()]).transpose()
Missing.columns = ['count']
Missing.sort_values(by=['count'], ascending=False)

# There are several columns which are mostly nulls.

,count
dti_joint,886870
annual_inc_joint,886868
verification_status_joint,886868
il_util,868762
mths_since_rcnt_il,866569
...,...
total_pymnt,0
total_pymnt_inv,0
total_rec_prncp,0
total_rec_int,0


In [120]:
# I am going to exclude the columns that are missing more than 50000 of their values.

# Missing.columns=['count']
#Missing.sort_values(by='count', ascending=False)

#Finding out the list of columns with less than 50000 null values

Missing_50 = Missing[Missing['count'] < 50000]
Subset_50 = Missing_50.index
print(Subset_50)

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'loan_status', 'pymnt_plan', 'url', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq'],
      dtype='object')


In [121]:
#subsetting the columns with less than 50000 null values

Loan_50 = Loan_data[Subset_50]
Loan_50.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,0.00,0.00,0.00,Jan-2015,171.62,Jan-2016,0.0,1.0,INDIVIDUAL,0.0
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,0.00,0.00,0.00,Jun-2014,649.91,Jan-2016,0.0,1.0,INDIVIDUAL,0.0
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,16.97,0.00,0.00,Jan-2015,357.48,Jan-2015,0.0,1.0,INDIVIDUAL,0.0
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,...,0.00,0.00,0.00,Jan-2016,67.79,Jan-2016,0.0,1.0,INDIVIDUAL,0.0


In [122]:
#Checking null values in the remining columns
#The highest null value is the emp_length column with 44825 nulls
#It sounds like an important column, so I want to keep it for now.

Loan_50.isnull().sum()

id                                0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
term                              0
int_rate                          0
installment                       0
grade                             0
sub_grade                         0
emp_length                    44825
home_ownership                    0
annual_inc                        4
verification_status               0
issue_d                           0
loan_status                       0
pymnt_plan                        0
url                               0
purpose                           0
title                           152
zip_code                          0
addr_state                        0
dti                               0
delinq_2yrs                      29
earliest_cr_line                 29
inq_last_6mths                   29
open_acc                         29
pub_rec                     

Checking out the remining columns to decide which columns I need to drop
based on their importance for the analysis. 
I will also consider if some columns present redundunt information.

In [123]:
print(Loan_50[['id', 'member_id']].head(5))

        id  member_id
0  1077501    1296599
1  1077430    1314167
2  1077175    1313524
3  1076863    1277178
4  1075358    1311748


In [124]:
# Similar information in 'purpose' and 'title' columns, so I will choose one of the two. 

Loan_50[['purpose', 'title']].value_counts()

purpose             title                               
debt_consolidation  Debt consolidation                      413228
credit_card         Credit card refinancing                 163954
home_improvement    Home improvement                         39951
other               Other                                    31774
debt_consolidation  Debt Consolidation                       14731
                                                             ...  
                    Debt Consolidation at a lower rate           1
                    Debt Consolidation debt                      1
                    Debt Consolidation etc                       1
                    Debt Consolidation for $150K+ earner         1
wedding             youngest daughter                            1
Length: 71255, dtype: int64

In [125]:
# Looking at the purpose column. No null values.

Loan_50['purpose'].value_counts()

debt_consolidation    524215
credit_card           206182
home_improvement       51829
other                  42894
major_purchase         17277
small_business         10377
car                     8863
medical                 8540
moving                  5414
vacation                4736
house                   3707
wedding                 2347
renewable_energy         575
educational              423
Name: purpose, dtype: int64

In [126]:
# 152 null values in the title column. The information in the title column is similar to the the purpose column. 
# But the title column lacks uniformity and has some null values. 

print(Loan_50['title'].isna().sum())
Loan_50['title'].value_counts()

152


Debt consolidation                     414001
Credit card refinancing                164331
Home improvement                        40112
Other                                   31892
Debt Consolidation                      15760
                                        ...  
New Heater                                  1
Looking to get out of debt                  1
Debt Conso/Pay off initial LC loan          1
Emergency Home Repair                       1
Credit Card/Auto Repair                     1
Name: title, Length: 63144, dtype: int64

I am choosing the 'purpose' column since it doesn't have missing values and has fewer categories.

In [127]:
# dti = debt to income ratio. The less dti ratio, the better. 
# A DTI of 43% is typically the highest ratio a borrower can have 
# and still get qualified for a mortgage, but lenders generally seek ratios of no more than 36%.
# source: https://www.investopedia.com/terms/d/dti.asp

Loan_50['dti'].value_counts()
print(Loan_50['dti'].isna().sum())

0


In [128]:
# 'pub_rec' - public records like bankruptcy, judgment, or a tax lien
print(Loan_50['pub_rec'].isna().sum())

29


In [129]:
# I am going to assume that the null values likely mean 0 pub_rec.
# Imputing the null values with 0 

Loan_50['pub_rec'] = Loan_50['pub_rec'].fillna(0)
Loan_50['pub_rec'].isna().sum()

C:\Users\meske\AppData\Local\Temp/ipykernel_19284/4129347833.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Loan_50['pub_rec'] = Loan_50['pub_rec'].fillna(0)


0

In [130]:
Loan_50['total_rec_prncp'].head()

#I am not sure what 'total_rec_prncp' mean. Therefore I am leaving out this column.

0     5000.00
1      456.46
2     2400.00
3    10000.00
4     2233.10
Name: total_rec_prncp, dtype: float64

In [131]:
# Comparing funded_amnt and funded_amnt_inv columns. 
# Using just the funded_amnt column is sufficient for my analysis.

Loan_50[['funded_amnt', 'funded_amnt_inv']].head()


,funded_amnt,funded_amnt_inv
0,5000.0,4975.0
1,2500.0,2500.0
2,2400.0,2400.0
3,10000.0,10000.0
4,3000.0,3000.0


In [132]:
# I will exclude this column because I don't know what f and w stand for.

Loan_50['initial_list_status'].value_counts()

f    456848
w    430531
Name: initial_list_status, dtype: int64

In [133]:
# It looks like the acc_now_delinq column shows how many accounts of this person are deliquent.

Loan_50['acc_now_delinq'].value_counts()

0.0     883236
1.0       3866
2.0        208
3.0         28
4.0          7
5.0          3
6.0          1
14.0         1
Name: acc_now_delinq, dtype: int64

In [134]:
Loan_50['earliest_cr_line'].head(5)

0    Jan-1985
1    Apr-1999
2    Nov-2001
3    Feb-1996
4    Jan-1996
Name: earliest_cr_line, dtype: object

In [135]:
Loan_50['application_type'].value_counts()

INDIVIDUAL    886868
JOINT            511
Name: application_type, dtype: int64

In [136]:
Loan_50.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'loan_status', 'pymnt_plan', 'url', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq'],
      dtype='object')

In [164]:
#Based on my exploration of the columns and their contents, I am selecting 
#the below columns which I think may be useful for the analysis.

col_to_select = ['id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'loan_status', 'pymnt_plan', 'purpose', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
       'collections_12_mths_ex_med',  'application_type',
       'acc_now_delinq']

# Columns I excluded due to redundent or unnecessary information
# 'member_id','url', 'title', 'policy_code'

In [165]:
#subsetting the col_to_select

df = Loan_50[col_to_select]

In [166]:
df.head(3)

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,...,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,application_type,acc_now_delinq
0,1077501,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,10+ years,...,861.07,0.0,0.00,0.00,Jan-2015,171.62,Jan-2016,0.0,INDIVIDUAL,0.0
1,1077430,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,< 1 year,...,435.17,0.0,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,INDIVIDUAL,0.0
2,1077175,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,10+ years,...,603.65,0.0,0.00,0.00,Jun-2014,649.91,Jan-2016,0.0,INDIVIDUAL,0.0


In [167]:
# The home_ownership column

df['home_ownership'].value_counts()

MORTGAGE    443557
RENT        356117
OWN          87470
OTHER          182
NONE            50
ANY              3
Name: home_ownership, dtype: int64

In [168]:
# Combining the values 'ANY' and 'OTHER' in the home_ownership column. 
# I will be replacing ANY with OTHER since OTHER is more meaningful.
df['home_ownership'] = df['home_ownership'].replace(to_replace ='ANY',
                 value ='OTHER')
df['home_ownership'].value_counts()

C:\Users\meske\AppData\Local\Temp/ipykernel_19284/3623015314.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['home_ownership'] = df['home_ownership'].replace(to_replace ='ANY',


MORTGAGE    443557
RENT        356117
OWN          87470
OTHER          185
NONE            50
Name: home_ownership, dtype: int64

In [169]:
# It seems like the date the loan was issued.

df['issue_d'].describe()

count       887379
unique         103
top       Oct-2015
freq         48631
Name: issue_d, dtype: object

In [170]:
# Convert issue_d column to date.

df['issue_d'] = pd.to_datetime(df['issue_d'])


#I had to run the above code first before I run the below with formatting
#because I was getting error value with difficulty converting one of the values
#when the formatting was added.

df['issue_d'] = pd.to_datetime(df['issue_d'], format='%m%Y')

C:\Users\meske\AppData\Local\Temp/ipykernel_19284/1495446107.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['issue_d'] = pd.to_datetime(df['issue_d'])
C:\Users\meske\AppData\Local\Temp/ipykernel_19284/1495446107.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['issue_d'] = pd.to_datetime(df['issue_d'], format='%m%Y')


In [171]:
df['issue_d'].value_counts()

# I am not sure if I will be using this column.

2015-10-01    48631
2015-07-01    45962
2015-12-01    44342
2014-10-01    38782
2015-11-01    37530
              ...  
2007-08-01       74
2007-07-01       63
2008-09-01       57
2007-09-01       53
2007-06-01       24
Name: issue_d, Length: 103, dtype: int64

In [172]:
df['loan_status'].value_counts()
# ** this would be my target variable.
# Defination: A charge-off or chargeoff is a declaration by a creditor 
# (usually a credit card account) that an amount of debt is unlikely to be collected.(Wikipedia)

Current                                                601779
Fully Paid                                             207723
Charged Off                                             45248
Late (31-120 days)                                      11591
Issued                                                   8460
In Grace Period                                          6253
Late (16-30 days)                                        2357
Does not meet the credit policy. Status:Fully Paid       1988
Default                                                  1219
Does not meet the credit policy. Status:Charged Off       761
Name: loan_status, dtype: int64

In [173]:
#combining the categories fully paid that are worded differently. The same with the charged off category.
#There is a category 'issued' in the df_1617 data, but not in df. I will leave that as is for now.

df['loan_status'] = df['loan_status'].replace(to_replace ='Does not meet the credit policy. Status:Fully Paid', value ='Fully Paid')
df['loan_status'] = df['loan_status'].replace(to_replace ='Does not meet the credit policy. Status:Charged Off', value ='Charged Off')
df['loan_status'].value_counts()

C:\Users\meske\AppData\Local\Temp/ipykernel_19284/341566470.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loan_status'] = df['loan_status'].replace(to_replace ='Does not meet the credit policy. Status:Fully Paid', value ='Fully Paid')
C:\Users\meske\AppData\Local\Temp/ipykernel_19284/341566470.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loan_status'] = df['loan_status'].replace(to_replace ='Does not meet the credit policy. Status:Charged Off', value ='Charged Off')


Current               601779
Fully Paid            209711
Charged Off            46009
Late (31-120 days)     11591
Issued                  8460
In Grace Period         6253
Late (16-30 days)       2357
Default                 1219
Name: loan_status, dtype: int64

In [174]:
df['term'].value_counts()

 36 months    621125
 60 months    266254
Name: term, dtype: int64

In [175]:
# There is a lot more data cleaning to be done which I will continue doing in the Exploratory Data Analysis stage.

In [176]:
df.to_csv('../data/interim/Loan_cleaned.csv',index=False)